<a href="https://colab.research.google.com/github/derewor/EMS-mutagenesis-project/blob/main/EMS_mutagenesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### EMS mutagenesis analysis code on BIGQUERY.
# project name= SEQURNCE ANALYSIS
# dataset name = emsmutagenesis
# This cte is cleaned data for all bugs.
WIth joined_table AS
(SELECT CASE
    WHEN REPLACE(REPLACE(h.position,'-','') ,'/','') is null AND REPLACE(REPLACE(j.position,'-',''),'/','') is null and REPLACE(REPLACE(g.position,'-',''),'/','') is null THEN 'L7D'
    WHEN REPLACE(REPLACE(d.position,'-',''),'/','') is null AND REPLACE(REPLACE(j.position,'-',''),'/','') is null and REPLACE(REPLACE(g.position,'-',''),'/','') is null THEN 'L21H'
    WHEN REPLACE(REPLACE(d.position,'-',''),'/','') is null AND REPLACE(REPLACE(h.position,'-',''),'/','') is null and REPLACE(REPLACE(g.position,'-',''),'/','') is null THEN 'L13J'
    WHEN REPLACE(REPLACE(d.position,'-',''),'/','') is null AND REPLACE(REPLACE(h.position,'-',''),'/','') is null and REPLACE(REPLACE(j.position,'-',''),'/','') is null THEN 'L17J'
            WHEN REPLACE(REPLACE(h.position,'-',''),'/','') is null and REPLACE(REPLACE(g.position,'-',''),'/','') is null THEN 'L7D_13J'
            WHEN REPLACE(REPLACE(j.position,'-',''),'/','') is null and REPLACE(REPLACE(h.position,'-',''),'/','') is null THEN 'L7D_17J'
            WHEN REPLACE(REPLACE(j.position,'-',''),'/','') is null and REPLACE(REPLACE(g.position,'-',''),'/','') is null THEN 'L7D_21H'
            WHEN REPLACE(REPLACE(d.position,'-',''),'/','') is null and REPLACE(REPLACE(h.position,'-',''),'/','') is null THEN 'L13J_17J'
            WHEN REPLACE(REPLACE(d.position,'-',''),'/','') is null and REPLACE(REPLACE(g.position,'-',''),'/','') is null THEN 'L13J_21H'
            WHEN REPLACE(REPLACE(d.position,'-',''),'/','') is null and REPLACE(REPLACE(j.position,'-',''),'/','') is null THEN 'L21H_17J'
                  WHEN REPLACE(REPLACE(d.position,'-',''),'/','') is null THEN 'L21H_17J_13J'
                  WHEN REPLACE(REPLACE(h.position,'-',''),'/','') is null THEN 'L7D_17J_13J'
                  WHEN REPLACE(REPLACE(g.position,'-',''),'/','') is null THEN 'L21H_7D_13J'
                  WHEN REPLACE(REPLACE(j.position,'-',''),'/','') is null THEN 'L21H_17J_7D'
                  ELSE 'ALL_PRESENCE' END as line_type,
                  RIGHT(d.chromosome, 1) as chrom1, CAST(REPLACE(REPLACE(d.position,'-',''),'/','') AS INT) as pos1, d. reference_seq AS ref_seq, d.alternative_seq As alt_seq,
                  RIGHT(h.chromosome, 1) as chrom2, CAST(REPLACE(REPLACE(h.position,'-',''),'/','') AS INT) as pos2, h.reference_seq AS ref_seq2, h.alternative_seq As alt_seq2,
                  RIGHT(g.chromosome, 1) as chrom3, CAST(REPLACE(REPLACE(g.position,'-',''),'/','') AS INT) as pos3, g.reference_seq AS ref_seq3, g.alternative_seq As alt_seq3,
                  RIGHT(j.chromosome, 1) as chrom4, CAST(REPLACE(REPLACE(j.position,'-',''),'/','') AS INT) as pos4, j.reference_seq AS ref_seq4, j.alternative_seq As alt_seq4
FROM `sequence-analysis-425810.emsmutagenesis.emsline7d` d
  FULL JOIN `sequence-analysis-425810.emsmutagenesis.emsline21h` h ON REPLACE(REPLACE(d.position,'-',''),'/','') = REPLACE(REPLACE(h.position,'-',''),'/','')
  FULL JOIN `sequence-analysis-425810.emsmutagenesis.emsline13j` j ON REPLACE(REPLACE(h.position,'-',''),'/','') = REPLACE(REPLACE(j.position,'-',''),'/','')
  FULL JOIN `sequence-analysis-425810.emsmutagenesis.emsline17j` g ON REPLACE(REPLACE(h.position,'-',''),'/','') = REPLACE(REPLACE(g.position,'-',''),'/','')

ORDER BY d.chromosome, d.position)

In [ ]:
## This is written to select SNPs specific to line L7D from the cte.
	SELECT *
	FROM joined_table jt
	WHERE line_type = 'L7D'
	ORDER BY chrom1, pos1;

In [ ]:
## This is written to select SNPs specific to line L13J from the cte.
	SELECT *
	FROM joined_table jt
	WHERE line_type = 'L13J'
	ORDER BY chrom4, pos4;

In [ ]:
## This is written to select SNPs specific to line L17J from the cte.
	SELECT *
	FROM joined_table jt
	WHERE line_type = 'L17J' AND chrom3 IN ('2','5')
	ORDER BY chrom3, pos3;

In [ ]:
## This is written to select SNPs specific to line 21H from the cte.
	SELECT *
	FROM joined_table jt
	WHERE line_type = 'L21H' AND chrom2 IN ('2','5')
	ORDER BY chrom2, pos2;

In [ ]:
##Further SNP and indel characterization of line 17J.

With extracted_L17J AS
	(SELECT line_type, chrom3, pos3, ref_seq3, alt_seq3
	 FROM `sequence-analysis-425810.emsmutagenesis.L17J`),
cleaned_L17J AS
(
	SELECT *
	FROM extracted_L17J
	WHERE ref_seq3 != 'N' or alt_seq3 != 'N')

	SELECT *
	FROM cleaned_L17J
	WHERE length(ref_seq3) = 1 and length(alt_seq3) = 1 AND chrom3 = 2;

In [ ]:
SELECT *
	FROM cleaned_L17J
	WHERE length(ref_seq3) = 1 and length(alt_seq3) = 1 AND chrom3 = 5;

In [ ]:
SELECT *, length(ref_seq3) AS ref_nucleo_num, length(alt_seq3) AS alt_nucleo_num, length(ref_seq3) -  length(alt_seq3) AS diff_nucl,
        	 CASE WHEN length(ref_seq3) -  length(alt_seq3) > 0 THEN 'DELETION'
            	WHEN length(ref_seq3) -  length(alt_seq3) < 0 THEN 'INSERTION'
            	ELSE 'UNCHANGED'
            	END AS indel
	FROM cleaned_L17J
	WHERE length(ref_seq3) > 1 and length(alt_seq3) > 1 AND chrom3 = 2;

In [ ]:
SELECT *, length(ref_seq3) AS ref_nucleo_num, length(alt_seq3) AS alt_nucleo_num, length(ref_seq3) -  length(alt_seq3) AS diff_nucl,
         	CASE WHEN length(ref_seq3) -  length(alt_seq3) > 0 THEN 'DELETION'
            	WHEN length(ref_seq3) -  length(alt_seq3) < 0 THEN 'INSERTION'
            	ELSE 'UNCHANGED'
            	END AS indel
	FROM cleaned_L17J
	WHERE length(ref_seq3) > 1 and length(alt_seq3) > 1 AND chrom3 = 5;

In [ ]:
##Further SNP and INDEL Sequence characterization of line 21H

With extracted_L21H AS
	(SELECT line_type, chrom2, pos2, ref_seq2, alt_seq2
	 FROM `sequence-analysis-425810.emsmutagenesis.L21H`),
cleaned_L21H AS
(
	SELECT *
	FROM extracted_L21H
	WHERE ref_seq2 != 'N' or alt_seq2 != 'N')
SELECT *
	FROM cleaned_L21H
	WHERE length(ref_seq2) = 1 and length(alt_seq2) = 1 AND chrom2 = 2

In [ ]:
SELECT *
	FROM cleaned_L21H
	WHERE length(ref_seq2) = 1 and length(alt_seq2) = 1 AND chrom2 = 2

In [ ]:
SELECT *
	FROM cleaned_L21H
	WHERE length(ref_seq2) = 1 and length(alt_seq2) = 1 AND chrom2 = 5

In [ ]:
SELECT *, length(ref_seq2) AS ref_nucleo_num, length(alt_seq2) AS alt_nucleo_num, length(ref_seq2) -  length(alt_seq2) AS diff_nucl,
	         CASE WHEN length(ref_seq2) -  length(alt_seq2) > 0 THEN 'DELETION'
	         WHEN length(ref_seq2) -  length(alt_seq2) < 0 THEN 'INSERTION'
	         ELSE 'UNCHANGED'
            	 END AS indel
	FROM cleaned_L21H
	WHERE length(ref_seq2) > 1 and length(alt_seq2) > 1 AND chrom2 = 2


In [ ]:
SELECT *, length(ref_seq2) AS ref_nucleo_num, length(alt_seq2) AS alt_nucleo_num, length(ref_seq2) -  length(alt_seq2) AS diff_nucl,
		CASE WHEN length(ref_seq2) -  length(alt_seq2) > 0 THEN 'DELETION'
	        WHEN length(ref_seq2) -  length(alt_seq2) < 0 THEN 'INSERTION'
            	ELSE 'UNCHANGED'
            	END AS indel
	FROM cleaned_L21H
	WHERE length(ref_seq2) > 1 and length(alt_seq2) > 1 AND chrom2 = 5